# Calculate Summary Data

Calculate a few summary datasets from the web-scrapped full list of corgs.

In [2]:
import json
import numpy as np
import pandas as pd
import csv

In [3]:
saveFileDir = '/Users/jillnaiman/spring2020/week12/corg/'

In [4]:
saveFilejsonLarge = saveFileDir + 'corgiData_countries_full_2020.json'

with open(saveFilejsonLarge) as json_file:
    data = json.load(json_file)

In [5]:
data[1]

{'name': 'A Blue Day Pleasure from The Small Hill Rebels',
 'dam': 'Bonnie from the Stone of Scone',
 'sire': 'BlÃ¥tirans Gandalf The Gray',
 'sex': 'Male',
 'year': '2013',
 'countries': 'Germany',
 'siblings': ['Adam Atom from The Small Hill Rebels',
  'Aery Arya from The Small Hill Rebels',
  'Alert Annie from The Small Hill Rebels',
  'All tomorrows parties from The Small Hill Rebels',
  'Allistair Applepie from The Small Hill Rebels',
  'Always Alwin from The Small Hill Rebels',
  'Arctic Aeryn from The Small Hill Rebels',
  'Ask for Aron from The Small Hill Rebels'],
 'country_id': 'DEU'}

Grab all unique country codes from what is here:

In [6]:
df = pd.read_csv(saveFileDir + 'world_population.tsv', sep="\t")   # read dummy .tsv file into memory

# convert to array
import numpy as np
dataCountry = np.array(df)

country_codes = dataCountry[:,0]
country_names = dataCountry[:,1]

In [7]:
country_codes

array(['CHN', 'IND', 'USA', 'IDN', 'BRA', 'PAK', 'BGD', 'NGA', 'RUS',
       'JPN', 'MEX', 'PHL', 'VNM', 'ETH', 'DEU', 'EGY', 'TUR', 'COD',
       'IRN', 'THA', 'FRA', 'GBR', 'ITA', 'MMR', 'ZAF', 'KOR', 'UKR',
       'COL', 'SDN', 'TZA', 'ARG', 'ESP', 'KEN', 'POL', 'DZA', 'CAN',
       'UGA', 'MAR', 'PER', 'IRQ', 'SAU', 'AFG', 'NPL', 'UZB', 'VEN',
       'MYS', 'GHA', 'YEM', 'TWN', 'PRK', 'SYR', 'ROU', 'MOZ', 'AUS',
       'LKA', 'MDG', 'CIV', 'CMR', 'NLD', 'CHL', 'BFA', 'NER', 'KAZ',
       'MWI', 'ECU', 'KHM', 'SEN', 'MLI', 'GTM', 'AGO', 'ZMB', 'ZWE',
       'CUB', 'RWA', 'GRC', 'PRT', 'TUN', 'TCD', 'BEL', 'GIN', 'CZE',
       'SOM', 'BOL', 'HUN', 'BDI', 'DOM', 'BLR', 'HTI', 'SWE', 'BEN',
       'AZE', 'AUT', 'HND', 'CHE', 'TJK', 'ISR', 'SRB', 'BGR', 'HKG',
       'LAO', 'LBY', 'JOR', 'PRY', 'TGO', 'PNG', 'SLV', 'NIC', 'ERI',
       'DNK', 'KGZ', 'SVK', 'FIN', 'SLE', 'ARE', 'TKM', 'CAF', 'SGP',
       'NOR', 'BIH', 'GEO', 'CRI', 'HRV', 'MDA', 'NZL', 'IRL', 'COG',
       'LBN', 'PRI',

## Number of corgis in each country as a function of time

In [8]:
# min/max time
minTime = 100000
maxTime = -1

for d in data:
    if len(d['year']) > 0: # empty entries
        if (d['year'].isdigit()) and (len(d['year']) == 4) and (int(d['year'])>1900): # formatting
            if (int(d['year']) < minTime):
                minTime = int(d['year'])
            if int(d['year']) > maxTime:
                maxTime = int(d['year'])

In [9]:
minTime,maxTime

(1917, 2020)

In [10]:
dates = np.arange(minTime, maxTime+1)
#dates

Loop & fill:

In [11]:
corgs = np.zeros([len(dates), len(country_codes)]) # count corgs

In [12]:
for d in data:
    if len(d['year']) > 0: # empty entries
        if (d['year'].isdigit()) and (len(d['year']) == 4) and (len(d['country_id']) > 0) and (int(d['year']) > 1900): # formatting
            year = int(d['year'])
            corgs[ dates == year, country_codes == d['country_id']] += 1

In [13]:
corgs.max(), corgs.min(), corgs.sum()

(621.0, 0.0, 36728.0)

Create json entries, ordered by country:

In [14]:
# # this one is too fancy
# v = []
# v.append({"years":dates.tolist()})

# vc = []
# for i in range(len(country_codes)):
#     vc.append( {"country":country_names[i], "country_id":country_codes[i], "num_corgs":corgs[:, i].tolist()} )

# # put all together
# v.append( {"countryInfo": vc} )

v = []
for i in range(len(dates)):
    countries = {'year': str(int(dates[i]))}
    for j in range(len(country_names)):
        countries[country_names[j]] = corgs[i,j]
    v.append(countries)

In [15]:
#v

In [16]:
saveFilejson = saveFileDir + 'corgs_per_country_over_time_2020.json'
f = open(saveFilejson,'w')
f.write(json.dumps(v,indent=2))
f.close()

Also save as CSV, for completeness:

In [17]:
csvFileName = saveFileDir + 'corgs_per_country_over_time_2020.csv'

header = ["", 'Country', 'CountryID']
# add in each year
for i in range(len(dates)):
    header.append('y'+str(dates[i]))
    
    
# formatting for years
years = ["", "YEARS", "YEARS"]
for i in range(len(dates)):
    years.append(str(dates[i]))

with open(csvFileName, 'w') as csvFile:
    writer = csv.writer(csvFile, quoting=csv.QUOTE_ALL)
    writer.writerow(header)
    writer.writerow(years)
    for i in range(len(country_codes)):
        row = [i, str(country_names[i]), country_codes[i]]
        for j in range(len(dates)):
            row.append(corgs[j,i].astype('int'))
            
        writer.writerow(row)
    csvFile.close()


In [18]:
# now, other organization

haveVals = []

# check non-zeros
for i in range(len(country_codes)):
    v = corgs[:,i] # all dates
    if v.sum() > 0:
        haveVals.append(True)
    else:
        haveVals.append(False)

In [19]:
# make header of non-zero country names
header = ["Years"]
for i in range(len(country_codes)):
    if haveVals[i]:
        header.append(country_names[i])

In [20]:
# loop and fill
csvFileName = saveFileDir + 'corgs_per_country_over_time_columns_2020.csv'

with open(csvFileName, 'w') as csvFile:
    writer = csv.writer(csvFile, quoting=csv.QUOTE_ALL)
    writer.writerow(header)
    for i in range(len(dates)):
        row = [dates[i]]
        for j in range(len(country_codes)):
            if haveVals[j]:
                row.append(corgs[i,j].astype('int'))
            
        writer.writerow(row)
    csvFile.close()


In [25]:
# save totals for each country too

v = []
for i in range(len(country_names)):
    totalCorg = np.sum(corgs[:,i])
    data = {'country_id': country_codes[i], 'totalCorg':totalCorg}
    v.append(data)
    
saveFilejson = saveFileDir + 'corgs_per_country_2020.json'
f = open(saveFilejson,'w')
f.write(json.dumps(v,indent=2))
f.close()

In [22]:
#corgs[dates, country]
corgs.shape

(104, 242)

In [23]:
np.sum(corgs, axis=0).shape

(242,)

In [24]:
len(country_names)

242